In [12]:
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [13]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [14]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [15]:
ch = Website("https://chandana-6.github.io/ChandanaPages/")
ch.get_contents()
ch.links

['https://www.linkedin.com/in/chandanasomavarapu/',
 '#header',
 'index.html',
 '#',
 '#',
 'https://github.com/Chandana-6/AdventureWorksSalesAnalysis',
 '#',
 '#',
 '#',
 '#',
 'https://github.com/Chandana-6/PortfolioProjectsSql',
 '#',
 '#',
 'https://github.com/Chandana-6/FAKE-NEWS-DETECTION',
 '#',
 '#',
 'https://github.com/Chandana-6/Scrabble',
 '#',
 '#',
 'https://www.linkedin.com/in/chandanasomavarapu/',
 '#']

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [16]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [17]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [18]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [20]:
print(get_links_user_prompt(ch))

Here is the list of links on the website of https://chandana-6.github.io/ChandanaPages/ - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://www.linkedin.com/in/chandanasomavarapu/
#header
index.html
#
#
https://github.com/Chandana-6/AdventureWorksSalesAnalysis
#
#
#
#
https://github.com/Chandana-6/PortfolioProjectsSql
#
#
https://github.com/Chandana-6/FAKE-NEWS-DETECTION
#
#
https://github.com/Chandana-6/Scrabble
#
#
https://www.linkedin.com/in/chandanasomavarapu/
#


In [21]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [22]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/nvidia/parakeet-tdt-0.6b-v2',
 '/Wan-AI/Wan2.1-VACE-14B',
 '/nari-labs/Dia-1.6B',
 '/multimodalart/isometric-skeumorphic-3d-bnb',
 '/lodestones/Chroma',
 '/models',
 '/spaces/enzostvs/deepsite',
 '/spaces/smolagents/computer-agent',
 '/spaces/Lightricks/ltx-video-distilled',
 '/spaces/ByteDance/DreamO',
 '/spaces/NihalGazi/FLUX-Pro-Unlimited',
 '/spaces',
 '/datasets/openbmb/Ultra-FineWeb',
 '/datasets/PrimeIntellect/INTELLECT-2-RL-Dataset',
 '/datasets/nvidia/OpenCodeReasoning',
 '/datasets/nvidia/OpenMathReasoning',
 '/datasets/DMindAI/DMind_Benchmark',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Writer',


In [23]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'models page', 'url': 'https://huggingface.co/models'},
  {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'},
  {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [24]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [25]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Posts
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Brow

In [26]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

In [27]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [28]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nPosts\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nnvidia/parakeet-tdt-0.6b-v2\nUpdated\n4 days ago\n•\n56.6k\n•\n977\nWan-AI/Wan2.1-VACE-14B\nUpdated\n1 day ago\n•\n8.8k\n•\n198\nnari-labs/Dia-1.6B\nUpdated\n6 days ago\n•\n143k\n•\n2.27k\nmultimodalart/isometric-skeumorphic-3d-bnb\nUpdated\n5 days ago\n•\n550\n•\n190\nlodestones/Chroma\nUpdated\n1 day ago\n•\n618\nBrowse 1M+ models\nSpaces\nRunning\n6.79k\n6.79k\nDeepSite\n🐳\nGenerate any app

In [30]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [31]:
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community discussion page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face Brochure

## Welcome to Hugging Face

**The AI Community Building the Future**

At Hugging Face, we empower the machine learning community to collaborate, innovate, and create. Our platform is designed for developers, researchers, and AI enthusiasts to discover, train, and share state-of-the-art models and datasets. With a mission to democratize good machine learning, we provide the tools and community support needed to build the future of AI.

---

## Our Offerings

### Models & Datasets
- **1M+ Models**: Access a vast library of cutting-edge models, including text, image, and audio recognition.
- **250k+ Datasets**: Browse and utilize diverse datasets for training and research purposes.

### Cutting-Edge Technology
- **Open Source Stack**: Leverage the Hugging Face library and tools, such as Transformers, Diffusers, and Tokenizers, to enhance your machine learning journey.
- **Inference Endpoints**: Deploy and scale your applications effortlessly with optimized solutions starting at just $0.60/hour for GPU access.

### Enterprise Solutions
Join over 50,000 organizations, including major players like Google, Amazon, and Microsoft, who trust Hugging Face for enterprise-grade AI solutions. Our platform provides security, dedicated support, and access controls tailored to your needs.

---

## Company Culture

At Hugging Face, we foster a **collaborative and inclusive environment**. Our growing team of 216 dedicated members is passionate about machine learning and works together to push the boundaries of AI technology. 

- **Community-Driven**: We believe in the power of the community, encouraging sharing, collaboration, and open dialogue.
- **Commitment to Learning**: Continuous professional development is emphasized through team activities, workshops, and resources for growth.

---

## Careers at Hugging Face

Are you passionate about AI and looking for a place to make a difference? Join us at Hugging Face! We are constantly on the lookout for innovative minds to contribute to our mission. Check our [Careers Page](https://huggingface.co/jobs) for open positions in engineering, research, community management, and more.

---

## Get Involved

### Explore & Collaborate
- **Spaces**: Create your own ML applications with "Spaces" where you can run and share your models.
- **Join Our Community**: Connect with fellow machine learning enthusiasts on platforms like Discord, Twitter, and our Forum.

### Start Your Journey!
Ready to dive into the world of AI? Sign up today and start collaborating with the Hugging Face community!
- **[Sign Up](https://huggingface.co/signup)**
- **[Explore Our Models](https://huggingface.co/models)**

---

## Follow Us

Stay updated on our latest releases, blog articles, and community activities through our social media platforms:
- **Twitter**: [@huggingface](https://twitter.com/huggingface)
- **LinkedIn**: [Hugging Face](https://www.linkedin.com/company/huggingface)
- **GitHub**: [huggingface](https://github.com/huggingface)

---

**Join us today in building the future of AI!**

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [32]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [33]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'discussion page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face Brochure

## Welcome to Hugging Face
### The AI Community Building the Future

Hugging Face is at the forefront of the AI revolution, serving as a collaborative platform where machine learning enthusiasts come together to create, discover, and share models, datasets, and applications. With over 1 million models and hundreds of thousands of datasets available, we provide the tools to accelerate your machine learning projects.

---

## Our Offerings
- **Models**: Access and collaborate on a wide variety of machine learning models designed for various tasks across multiple modalities, including text, image, video, audio, and even 3D.
- **Datasets**: Explore and share more than 250,000 datasets curated for diverse ML needs.
- **Spaces**: Run and showcase innovative applications powered by machine learning.
- **Enterprise Solutions**: Our enterprise-grade tools ensure security, access controls, and dedicated support for teams looking to leverage AI at scale.

### Featured Services
- **Compute**: Deploy and manage optimized inference endpoints with pricing starting at $0.60/hour.
- **Enterprise Plans**: Tailored plans for teams, starting at $20/user/month, offering priority support and resource management features.

---

## Who We Serve
Hugging Face is trusted by over 50,000 organizations, including industry leaders like:
- **Google**
- **Microsoft**
- **Amazon**
- **Meta**
- **Intel**
- **Grammarly**

Join a community of innovators who rely on Hugging Face to transform their AI initiatives.

---

## Culture and Community
At Hugging Face, our mission is to democratize machine learning. We embrace openness, creativity, and continual learning. As a community-driven platform, we encourage collaboration and the sharing of knowledge.

- **Open Source Commitment**: We work together with the community to build and improve the tools that drive machine learning forward, including popular libraries like Transformers and Diffusers.
- **Diversity and Inclusion**: We value diverse perspectives and strive to create an inclusive space for all contributors.

### Join Us
Be a part of the AI community! Hugging Face is always looking for talent to join our teams. Whether you’re an engineer, researcher, or designer, we invite you to help shape the future of machine learning.

**Explore our job openings** on our [Jobs Page](https://huggingface.co/jobs).

---

## Connect with Us
Stay updated with the latest developments and join our discussions:
- **Website**: [Hugging Face](https://huggingface.co)
- **GitHub**: [Hugging Face on GitHub](https://github.com/huggingface)
- **Twitter**: [@huggingface](https://twitter.com/huggingface)
- **Discord**: Join our community on Discord

--- 

Together, we can build a future where machine learning is accessible to everyone. Welcome to Hugging Face!

In [34]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face Brochure

## Introduction
**Hugging Face** is a leading AI community committed to building the future through collaboration in machine learning. By providing an innovative platform where developers and researchers can work together on models, datasets, and applications, Hugging Face is at the forefront of the AI revolution.

---

## Our Offerings
### Models
With **over 1 million models** available, Hugging Face enables users to explore numerous machine learning applications. Some trending models include:
- **nvidia/parakeet-tdt-0.6b-v2**
- **Wan-AI/Wan2.1-VACE-14B**
- **multimodalart/isometric-skeumorphic-3d-bnb**

### Datasets
Access and share **more than 250k datasets** to support a variety of machine learning tasks. Unlock the potential of your ML projects with high-quality datasets curated from the community.

### Spaces
Engage with over **400k AI applications** hosted on our platform, including innovative solutions like **DeepSite** and **DreamO** for application generation and image customization.

### Enterprise Solutions
Hugging Face provides a paid Compute and Enterprise solution starting at **$20/user/month**, designed to meet the needs of over **50,000 organizations**, including giants like Google, Amazon, and Microsoft. Offerings include:
- **Single Sign-On**
- **Priority Support**
- **Resource Groups**

---

## Company Culture
At Hugging Face, we celebrate an **open-source mentality**, encouraging collaboration and transparency within the AI community. Our team thrives on innovation, with a focus on personal growth and the sharing of knowledge. We value diverse perspectives and promote an inclusive environment that fosters creativity and collective problem-solving.

---

## Career Opportunities
Join the Hugging Face team and be part of a vibrant community dedicated to building cutting-edge AI technologies. We seek passionate individuals who are eager to contribute to collaborative projects and push the boundaries of machine learning. Explore our **current job openings** on our website to see how you can make an impact with us!

---

## Join Us
Whether you are a developer, researcher, or simply an AI enthusiast, *Hugging Face* welcomes you to become part of our mission. Connect with the community, explore new technologies, and build the future of AI together.

[Sign Up](https://huggingface.co/join) to start your journey today! 

---

**Contact Information:**
- **Website:** [huggingface.co](https://huggingface.co)
- **Social:** Follow us on [GitHub](https://github.com/huggingface), [Twitter](https://twitter.com/huggingface), [LinkedIn](https://www.linkedin.com/company/huggingface), and [Discord](https://discord.gg/huggingface).

---

*Let’s build the future of machine learning collaboratively!*
